In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

In [4]:
train_set = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('../data', train=False, transform=transform)

In [5]:
def ConvBlock(first_chanels, second_chanels): # (batch, 512, ...)
    return nn.Sequential(
            nn.Conv2d(second_chanels, first_chanels, 1), # 512, 256
            nn.BatchNorm2d(first_chanels),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(first_chanels, second_chanels, 3, padding=1), # 256, 512
            nn.BatchNorm2d(second_chanels),
            nn.LeakyReLU(0.1, inplace=True),
        )

In [6]:
class Net(nn.Module):
    def __init__(self, num_classes, drop_rate):
        super(Net, self).__init__()
        self.num_classes = num_classes
        self.drop_rate = drop_rate
        
        self.conv1 = nn.Conv2d(1, 16, 5, stride=2, padding=2) # (batch, 16, 14, 14)
        self.lrelu1 = nn.LeakyReLU(0.1, inplace=True)
        self.bn1 = nn.BatchNorm2d(16)
        self.convb1 = ConvBlock(8, 16) # (batch, 16, 7, 7)
        self.maxp1 = nn.MaxPool2d(2)  # (batch, 16, 7, 7)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # (batch, 32, 7, 7)
        self.lrelu2 = nn.LeakyReLU(0.1, inplace=True)
        self.bn2 = nn.BatchNorm2d(32)
        self.convb1 = ConvBlock(16, 32)
        self.maxp2 = nn.MaxPool2d(2) # (batch, 32, 4, 4)

        self.fc1 = nn.LazyLinear(1000)
        self.relu = nn.ReLU(1000)
        self.dropout = nn.Dropout(self.drop_rate)
        self.fc2 = nn.Linear(1000, self.num_classes)
        
        
    def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)


    def forward(self, x):
        batch, _, _, _ = x.size()
        x = self.conv1(x)
        x = self.lrelu1(x)
        x = self.bn1(x)
        #x = self.convb1(x)
        x = self.maxp1(x)

        x = self.conv2(x)
        x = self.lrelu2(x)
        x = self.bn2(x)
        #x = self.convb2(x)
        x = self.maxp2(x)

        x = x.view(batch, -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
model = Net(10, 0.1)

C:\Users\da4nik\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [10]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=0)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

In [12]:
scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 200 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.439584
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.237579
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.766334
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.549575
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.188923
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.990432
Train Epoch: 1 [768/60000 (1%)]	Loss: 0.975524
Train Epoch: 1 [896/60000 (1%)]	Loss: 0.691588
Train Epoch: 1 [1024/60000 (2%)]	Loss: 0.578348
Train Epoch: 1 [1152/60000 (2%)]	Loss: 0.461841
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.362710
Train Epoch: 1 [1408/60000 (2%)]	Loss: 0.366968
Train Epoch: 1 [1536/60000 (3%)]	Loss: 0.551778
Train Epoch: 1 [1664/60000 (3%)]	Loss: 0.246402
Train Epoch: 1 [1792/60000 (3%)]	Loss: 0.412694
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.437551
Train Epoch: 1 [2048/60000 (3%)]	Loss: 0.354323
Train Epoch: 1 [2176/60000 (4%)]	Loss: 0.335286
Train Epoch: 1 [2304/60000 (4%)]	Loss: 0.303278
Train Epoch: 1 [2432/60000 (4%)]	Loss: 0.291591
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.323369
Tr

KeyboardInterrupt: 